# MAMOB - Trend Following Multi-Assets (CAC-like)

**Objectif** : construire un backtest *trend following* **long-only** multi-actifs, comparer **1 couple** vs **3 couples** de moyennes mobiles, et rendre l’évaluation **méthodologiquement robuste** via :
- suppression du *look-ahead bias* (`shift(1)`)
- contrainte **fully invested (0 cash)** via normalisation et fallback
- métriques **risk-adjusted** (CAGR, vol, Sharpe, MDD, turnover)
- tests de robustesse : **sensibilité aux paramètres** + (**option**) walk-forward IS/OOS

**Conventions** : données daily, annualisation sur **252** jours, Sharpe avec **rf = 0**.

> Ce notebook présente la réflexion, la méthodologie, et les preuves empiriques principales.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

plt.rcParams["figure.figsize"] = (11, 4)
pd.set_option("display.max_columns", 80)
pd.set_option("display.width", 220)


## 1) Research question & protocole

### Research question
Est-ce qu’une stratégie trend-following simple basée sur moyennes mobiles, appliquée à un univers multi-actifs (CAC-like), produit une performance **risk-adjusted** supérieure à un benchmark **Buy & Hold equal-weight** ?

### Hypothèses testées
- **H1** : le signal de trend (MA_short > MA_long) améliore le Sharpe vs Buy & Hold.
- **H2** : la performance n’est pas un artefact de sur-optimisation : elle reste raisonnablement stable sur des variantes proches des paramètres (sensibilité).
- **H3** : 1 couple (ex: 20/100) peut être plus robuste/pratique que 3 couples (moins de degrés de liberté, turnover potentiellement plus faible) — et cela doit se voir dans les métriques.

### Règles de backtest 
1) **No look-ahead** : on trade à *t+1* un signal calculé à *t* via `shift(1)`.
2) **No cash (fully invested)** : contrainte \(\sum_i w_i(t)=1\) imposée à chaque date.
3) **Comparaison complète** : on compare Trend vs Buy & Hold via rendement **et** risque (CAGR, vol, Sharpe, MDD) + turnover (proxy de churn/coûts).


## 2) Données

Par souci de clarté, on illustre sur un sous-univers (8 titres). L’application Flask associée permet de sélectionner un univers plus large.



In [ ]:
tickers = ["AI.PA","MC.PA","OR.PA","BNP.PA","SAN.PA","SU.PA","DG.PA","CAP.PA"]

raw = yf.download(
    tickers,
    period="15y",
    auto_adjust=False,
    progress=False,
    group_by="column"
)

prices = raw["Close"].dropna(how="all")
prices.tail()

In [ ]:
returns = prices.pct_change().fillna(0)
returns.tail()

## 3) Stratégie : signal MA + anti look-ahead

### Signal
- Signal haussier si `MA_short > MA_long` (=1)
- Sinon 0

### Pourquoi `shift(1)` ?
Sans `shift(1)`, on utiliserait la clôture de *t* pour décider et exécuter à *t*, ce qui induit un biais look-ahead. Avec `shift(1)`, le signal calculé à t s’applique à t+1, ce qui permet un backtest méthodologiquement propre.


In [ ]:
def ma_signal(prices: pd.DataFrame, short_w: int, long_w: int) -> pd.DataFrame:
    """Binary MA crossover signal, shifted by 1 day to avoid look-ahead."""
    ma_s = prices.rolling(short_w).mean()
    ma_l = prices.rolling(long_w).mean()
    sig = (ma_s > ma_l).astype(int)
    return sig.shift(1).fillna(0)

signal_20_100 = ma_signal(prices, 20, 100)
signal_20_100.tail()

## 4) Portefeuille **fully invested (0 cash)** : normalisation et fallback

Le signal fournit une **exposition relative** par actif (0/1 ; ou en tiers avec 3 couples). Pour obtenir un portefeuille, on impose la contrainte :

\[
\sum_i w_i(t) = 1
\]

### Normalisation
On normalise les expositions chaque jour :
\[
w_i(t) = \frac{exposure_i(t)}{\sum_j exposure_j(t)}
\]

### Cas critique (si tout bearish)
Si \(\sum exposure(t)=0\), la normalisation est impossible. Sous l’hypothèse **long-only fully invested**, on choisit le fallback suivant :
- **conserver les poids de la veille** (réduit le churn)
- au tout début (pas de veille) : **equal-weight**.

> Ce choix respecte l’hypothèse “0 cash” tout en évitant des bascules brutales.


In [ ]:
def normalize_weights_no_cash_with_fallback(exposure: pd.DataFrame) -> pd.DataFrame:
    """Normalize exposures to fully invested weights; fallback when all exposures are 0."""
    exposure = exposure.clip(lower=0)
    row_sum = exposure.sum(axis=1)
    w = exposure.div(row_sum.replace(0, np.nan), axis=0)
    w = w.ffill()  # keep last weights if sum=0
    w = w.fillna(1.0 / exposure.shape[1])  # initial equal-weight
    return w

weights_1 = normalize_weights_no_cash_with_fallback(signal_20_100)
weights_1.sum(axis=1).tail()

## 5) Backtest Trend (1 couple) + benchmark Buy & Hold

On calcule :
\[
r_p(t) = \sum_i w_i(t)\, r_i(t)
\]
et l’equity :
\[
Equity(t)=\prod_{u \le t}(1+r_p(u))
\]

### Benchmark
**Buy & Hold equal-weight** : poids constants \(1/N\) (pas de rebalancing explicite), portefeuille fully invested.


In [ ]:
def buy_hold_equal_weight(returns: pd.DataFrame) -> pd.Series:
    n = returns.shape[1]
    ew = np.repeat(1.0 / n, n)
    return (returns * ew).sum(axis=1)

port_ret_1 = (weights_1 * returns).sum(axis=1)
equity_1 = (1 + port_ret_1).cumprod()

bh_ret = buy_hold_equal_weight(returns)
bh_equity = (1 + bh_ret).cumprod()

pd.DataFrame({"Trend (20/100)": equity_1, "Buy&Hold EW": bh_equity}).dropna().plot(
    title="Equity curves — Trend 1-couple vs Buy & Hold"
)
plt.show()

## 6) 1 couple vs 3 couples (conviction en tiers)

### Construction “3 couples”
On prend 3 couples (fast/medium/slow), on calcule 3 signaux binaires, puis on calcule 
\[
exposure_i(t) = \frac{signal_1+signal_2+signal_3}{3} \in \{0,1/3,2/3,1\}
\]
On normalise ensuite pour rester fully invested.

On compare empiriquement Sharpe / MDD / turnover entre 1 et 3 couples.


In [ ]:
triple = [(12, 60), (24, 120), (48, 200)]

signals = [ma_signal(prices, s, l) for (s, l) in triple]
exposure_3 = sum(signals) / len(signals)

weights_3 = normalize_weights_no_cash_with_fallback(exposure_3)
port_ret_3 = (weights_3 * returns).sum(axis=1)
equity_3 = (1 + port_ret_3).cumprod()

pd.DataFrame({"Trend 1-couple": equity_1, "Trend 3-couples": equity_3, "Buy&Hold": bh_equity}).dropna().plot(
    title="Equity curves — 1-couple vs 3-couples (and Buy&Hold)"
)
plt.show()

## 7) KPIs : performance + risque + churn

On calcule :
- **Total return**
- **CAGR** (annualisé)
- **Vol annualisée**
- **Sharpe (rf=0)**
- **Max Drawdown (MDD)**
- **Turnover moyen** (proxy churn/coûts)


In [ ]:
def annualize_return(equity: pd.Series) -> float:
    if len(equity) < 2:
        return float("nan")
    total = equity.iloc[-1] / equity.iloc[0] - 1
    return (1 + total) ** (252 / len(equity)) - 1

def annualize_vol(ret: pd.Series) -> float:
    return float(ret.std() * np.sqrt(252))

def sharpe_ratio(ann_ret: float, ann_vol: float) -> float:
    if ann_vol == 0 or np.isnan(ann_vol):
        return float("nan")
    return float(ann_ret / ann_vol)

def max_drawdown(equity: pd.Series) -> float:
    peak = equity.cummax()
    dd = equity / peak - 1
    return float(dd.min())

def turnover_from_weights(weights: pd.DataFrame) -> pd.Series:
    return (weights - weights.shift(1).fillna(0)).abs().sum(axis=1)

def kpis(name: str, equity: pd.Series, ret: pd.Series, weights: pd.DataFrame) -> pd.Series:
    total = equity.iloc[-1] / equity.iloc[0] - 1
    ann = annualize_return(equity)
    vol = annualize_vol(ret)
    sh = sharpe_ratio(ann, vol)
    mdd = max_drawdown(equity)
    to_mean = float(turnover_from_weights(weights).mean())
    return pd.Series(
        {"Total Return": total, "CAGR": ann, "Vol": vol, "Sharpe": sh, "MaxDD": mdd, "Turnover_mean": to_mean},
        name=name
    )

# Buy&Hold weights (constant equal-weight matrix)
w_bh = pd.DataFrame(
    np.tile(1/returns.shape[1], (len(returns), returns.shape[1])),
    index=returns.index,
    columns=returns.columns
)

kpi_table = pd.concat([
    kpis("Trend 1-couple (20/100)", equity_1, port_ret_1, weights_1),
    kpis("Trend 3-couples (12/60|24/120|48/200)", equity_3, port_ret_3, weights_3),
    kpis("Buy&Hold EW", bh_equity, bh_ret, w_bh),
], axis=1).T

# pretty display
disp = kpi_table.copy()
for col in ["Total Return","CAGR","Vol","MaxDD"]:
    disp[col] = (disp[col] * 100).round(2)
disp["Sharpe"] = disp["Sharpe"].round(2)
disp["Turnover_mean"] = disp["Turnover_mean"].round(3)
disp

In [ ]:
plt.figure(figsize=(8,4))
plt.scatter(kpi_table["Turnover_mean"], kpi_table["Sharpe"])
for name in kpi_table.index:
    plt.annotate(name, (kpi_table.loc[name,"Turnover_mean"], kpi_table.loc[name,"Sharpe"]))
plt.xlabel("Average turnover (proxy costs)")
plt.ylabel("Sharpe")
plt.title("Trade-off: Sharpe vs Turnover")
plt.show()

## 8) Robustness check : sensibilité aux paramètres autour de 20/100

Même si 20/100 fonctionne, il faut vérifier qu’on n’est pas sur un point "chanceux". On teste une grille locale autour de (20, 100) et on observe la distribution des Sharpe/MDD/turnover.

Si 20/100 se situe au sein d’une zone de résultats cohérents, c’est un argument empirique fort de robustesse.


In [ ]:
candidates = []
for s in [10, 15, 20, 25, 30]:
    for l in [80, 90, 100, 110, 120]:
        if s < l:
            candidates.append((s,l))

rows = []
for s,l in candidates:
    expo = ma_signal(prices, s, l)
    w = normalize_weights_no_cash_with_fallback(expo)
    r = (w * returns).sum(axis=1)
    eq = (1+r).cumprod()
    ann = annualize_return(eq)
    vol = annualize_vol(r)
    rows.append({
        "pair": f"{s}/{l}",
        "CAGR": ann,
        "Vol": vol,
        "Sharpe": sharpe_ratio(ann, vol),
        "MaxDD": max_drawdown(eq),
        "Turnover": float(turnover_from_weights(w).mean())
    })

grid = pd.DataFrame(rows).sort_values("Sharpe", ascending=False).reset_index(drop=True)
grid.head(10)

In [ ]:
grid["Sharpe"].hist(bins=18)
plt.title("Sharpe distribution across MA candidates (sensitivity check)")
plt.xlabel("Sharpe")
plt.show()

## 9) Walk-forward IS/OOS (option)

L’application Flask associée implémente un **walk-forward** (type cross-validation temporelle) :
- split temporel : **70% IS** / **30% OOS**
- sélection du meilleur couple (ou triple) sur IS (ex: meilleur Sharpe IS)
- reporting de la performance sur OOS (vraie performance hors-échantillon)

Schéma conceptuel :
- **IS** : choix du paramètre \(\theta^* = \arg\max_{\theta} \text{Sharpe}_{IS}(\theta)\)
- **OOS** : évaluation \(\text{Perf}_{OOS}(\theta^*)\)

En complément, l’app calcule une **stabilité OOS en 3 blocs** (Sharpe bloc 1/2/3) pour détecter une performance dépendante d’une seule sous-période.


## 10) Difficultés rencontrées et solutions

1) **Look-ahead bias** : corrigé par `shift(1)`.
2) **Cash implicite** (somme des expositions < 1) : contrainte **fully invested** par normalisation.
3) **Régime bearish global** (tous signaux à 0) : fallback **keep last weights** + equal-weight au début.
4) **Risque de sur-optimisation** : sensibilité locale + (option) walk-forward IS/OOS dans l’app.
5) **Qualité / disponibilité des données** : sous-univers pour reproductibilité ; l’app supporte un univers plus large.


## 11) Conclusion

- backtest sans biais (shift(1))
- portefeuille **fully invested** (normalisation + fallback)
- comparaison Trend vs Buy&Hold via **CAGR/Vol/Sharpe/MDD/Turnover**
- robustesse : sensibilité aux paramètres + (option) walk-forward IS/OOS (dans l’app)

**Extensions naturelles** : intégrer des coûts explicites (slippage/fees), et tester sur univers plus large / périodes plus longues.
